In [24]:
from keras.applications import DenseNet121, ResNet50
from keras.layers import Input
from keras.models import Model
from keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2

In [17]:
target_size = (900, 400)


# Function to load and preprocess an image
def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.convert('RGB')
    img = img.resize(target_size[::-1], Image.Resampling.LANCZOS)
    # img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize pixel values to between 0 and 1
    return img_array


input_person_image = load_and_preprocess_image('person_image.png')  # Replace with the actual path
input_dress_image = load_and_preprocess_image('dress_image.png')  # Replace with the actual path

In [27]:
# Load a pre-trained segmentation model
model_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
segmentation_model = hub.load(model_url)

# Load and preprocess the input image
def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, (224, 224))  # Resize the image to match the model's input size
    return img

input_image_path = "person_image.png"
input_image = load_and_preprocess_image(input_image_path)

# Perform segmentation
input_image = tf.expand_dims(input_image, axis=0)
segmentation_result = segmentation_model(input_image)['segmentation_mask']
print(segmentation_result.shape)

# Post-process the segmentation result
segmentation_result = tf.image.resize(segmentation_result, (input_image.shape[1], input_image.shape[2]))
segmentation_result = tf.argmax(segmentation_result, axis=-1)
segmentation_result = segmentation_result[0].numpy()

# Create a mask for body parts
body_parts_mask = np.where(segmentation_result == 15, 255, 0)  # Adjust the class ID according to your model's output

# Apply the mask to the original image
input_image = cv2.imread(input_image_path)
segmented_image = cv2.bitwise_and(input_image, input_image, mask=body_parts_mask)

# Display the segmented image
cv2.imshow("Segmented Image", segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got 'segmentation_mask'

In [18]:
# Normalize and scale up the feature maps
def process_features(features, original_size):
    # Assuming we're using the first filter of the first layer
    feature_map = features[0, :, :, 0]

    # Normalize the feature map
    feature_map -= feature_map.mean()
    feature_map /= feature_map.std()
    feature_map *= 64
    feature_map += 128
    feature_map = np.clip(feature_map, 0, 255).astype('uint8')

    # Resize feature map to original size
    feature_map_image = Image.fromarray(feature_map)
    feature_map_image = feature_map_image.resize(original_size)
    return feature_map_image

In [19]:
person_input = Input(shape=(target_size[0], target_size[1], 3))
# Load ResNet with pre-trained weights, excluding top layers
resnet = ResNet50(include_top=False, weights='imagenet', input_tensor=person_input)

# Example: Adding a custom layer on top of ResNet
# You would extend this for parsing and pose estimation branches
x = resnet.output
# ... Add custom layers ...

# Create the model
model = Model(inputs=resnet.input, outputs=x)
model.compile(optimizer='adam', loss='mean_squared_error')
features_image_array = model.predict([input_person_image])
print(features_image_array.shape)
features_image_path = 'features_image.png'
processed_feature_map = process_features(features_image_array, target_size)
processed_feature_map.show()

1/1 [==============================] - 0s 417ms/step
(1, 29, 13, 2048)


/var/folders/3y/29x5vvy57135l3fqwbrn1l_80000gq/T/ipykernel_53818/3383339489.py:8: RuntimeWarning: invalid value encountered in divide
  feature_map /= feature_map.std()
/var/folders/3y/29x5vvy57135l3fqwbrn1l_80000gq/T/ipykernel_53818/3383339489.py:11: RuntimeWarning: invalid value encountered in cast
  feature_map = np.clip(feature_map, 0, 255).astype('uint8')


In [22]:
# Select the first image in the batch and remove the extra dimension
segmentation_map = np.squeeze(features_image_array, axis=0)

# Convert class probabilities to class labels
segmentation_map = np.argmax(segmentation_map, axis=-1)
segmentation_map_resized = np.array(Image.fromarray(segmentation_map).resize((target_size[1], target_size[0]), Image.Resampling.NEAREST))

# Apply a colormap for visualization (optional)
# The colormap helps differentiate the segments
plt.imshow(segmentation_map_resized, cmap='jet')  # You can choose a different colormap as per your preference
plt.colorbar()  # To show the color scale
plt.axis('off')

# Save or display the processed segmentation map
plt.savefig('segmentation_output.png', bbox_inches='tight', pad_inches=0)
# or display it
plt.show()

TypeError: Cannot handle this data type: (1, 1), <i8